<a href="https://colab.research.google.com/github/sungjindo/NLP/blob/main/Transformer_%EC%B1%97%EB%B4%87_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **필요 패키지 import**

In [1]:
!pip install tokenizers # Huggingface - 🤗 Tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.6 MB 32.3 MB/s 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import torch
import torch.nn as nn

# Transformer로 챗봇을 만들어봅시다!
- Transformer에 문장을 입력하면, 그에 대한 대답을 하는 챗봇을 만들 것입니다.
- 중간중간 빈칸을 채워넣어가며 자신만의 챗봇 코드를 완성해보세요.

우선 챗봇 학습을 위한 데이터를 다운로드 받습니다.

In [3]:
# data download
# from https://github.com/songys/Chatbot_data
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')

In [4]:
print('데이터 샘플의 개수 :', len(train_data))

데이터 샘플의 개수 : 11823


In [5]:
train_data.head(10)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


## 한국어 데이터를 전처리하는 방법에 대해 알아봅시다.
- 우리는 네이버 영화리뷰 데이터에 tokenizer를 학습시켜 단어를 구분하도록 할 예정입니다. 
(챗봇 데이터에 비해 훨씬 많은 데이터를 포함하고 있습니다.)

In [6]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
naver_df = pd.read_table('ratings.txt')
naver_df = naver_df.dropna(how='any')
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [7]:
from tokenizers import BertWordPieceTokenizer
tokenizer = BertWordPieceTokenizer(lowercase=False) # lowercase를 구분할지 여부를 선택합니다 (True: 대문자 무시(모두 소문자로 인식), False:대소문자 구분)

In [23]:
data_file = 'naver_review.txt'
vocab_size = 30000
limit_alphabet = 6000
min_frequency = 5 # 등장횟수, 

tokenizer.train(files=data_file,
                vocab_size=vocab_size, #사전의 사이즈- 전문용어같은 조금 나오는 단어도 있을 수 있다. 그런것은 사이즈를 설정해서 제외해버린다.!
                limit_alphabet=limit_alphabet, #알파벳의 개수, 많이 쓰이는 순서대로
                min_frequency=min_frequency,
                special_tokens=['[PAD]', '[START]', '[END]', '[UNK]'])

In [24]:
# vocab 저장
tokenizer.save_model('./')
vocab_df = pd.read_fwf('vocab.txt', header=None)
vocab_df

,0
0,[PAD]
1,[START]
2,[END]
3,[UNK]
4,!
...,...
29995,맘을
29996,맛도
29997,망하지
29998,망한다


In [26]:
vocab_df[vocab_df[0].map(lambda x : '##' in x)]

,0
3235,##0
3236,##분
3237,##은
3238,##랭
3239,##스
...,...
29767,##스페셜
29769,##주니어
29790,##마누라
29805,##아닙니다


sub-word : i lobe/ dog/ ##s/ cat 
   '##은 단어 앞에 무엇이 있었다는 뜻 


In [27]:
# 챗봇 모델은 토큰 인덱스를 이용하여 모델을 작동시키고, 최종 예측 결과도 마찬가지로 토큰 인덱스로 된 결과를 내뱉습니다.
# 인코딩된 결과물을 decode 함수를 통해 해석 가능한 문장으로 바꿔줄 수 있습니다.
encoded = tokenizer.encode('이 챗봇이 잘 완성될까요?')
print('토큰화 결과 :',encoded.tokens)
print('토큰 인덱스 :',encoded.ids)
print('디코딩 :',tokenizer.decode(encoded.ids))

토큰화 결과 : ['이', '챗', '##봇', '##이', '잘', '완성', '##될까', '##요', '?']
토큰 인덱스 : [2336, 2644, 4098, 3272, 2361, 13647, 20342, 3251, 34]
디코딩 : 이 챗봇이 잘 완성될까요?


In [28]:
vocab_df.loc[2336]

0    이
Name: 2336, dtype: object

In [11]:
tokenizer

Tokenizer(vocabulary_size=30000, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=False, wordpieces_prefix=##)

# Transformer 모델을 만들어봅시다.

In [29]:
from torch import Tensor
import torch
import torch.nn as nn
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)

        self.register_buffer('pos_embedding', pos_embedding) # register_buffer의 역할은 뭘까요?

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        # [1032, 512, 3, ...,] (batch size, sequence length)
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)
# word embedding * 큰숫자 + positional encoding => 순서정보보다 자체 정보가 더 중요해서 !

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = nn.Transformer(
                                       d_model=emb_size, #residule ~
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
      
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [12]:
model.state_dict()

NameError: ignored

- Transformer의 학습 중에는, 미래의 데이터를 볼 수 없도록 하는 마스크가 필요합니다.

In [14]:
PAD_IDX = 0
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0)) 
    return mask 


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len) # If a FloatTensor is provided, it will be added to the attention weight.
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool) #If a BoolTensor is provided, False values will be unchanged. 

    src_padding_mask = (src == PAD_IDX).transpose(0, 1) #If a BoolTensor is provided, positions with True are not allowed to attend.
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [15]:
SRC_VOCAB_SIZE = vocab_size # 30000
TGT_VOCAB_SIZE = vocab_size # 30000
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

# 모델을 initialize 해줍니다.
model = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
# cpu -> gpu
model = model.to(DEVICE) # .to(torch.device('cuda')) # .cuda()

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX) # ignore_index 가 하는 역할은 무엇일까요?

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, 
                             betas=(0.9, 0.98), 
                             eps=1e-9)

In [16]:
train_data.head(10)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


In [17]:
from torch.utils.data import Dataset
class ChatbotDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y, _ = self.data[idx]
        x_tokens = self.tokenizer.encode(x).ids
        y_tokens = self.tokenizer.encode(y).ids

        # [3,52,24,63] 
        x_tokens.insert(0,1)
        x_tokens.append(2)
        # [1, 3, 52, 24, 63, 2]
        # [<SOS>, ~~~~~~~~,  <EOS>]

        y_tokens.insert(0,1)
        y_tokens.append(2)

        return torch.tensor(x_tokens, dtype=torch.long), torch.tensor(y_tokens).long()

In [18]:
dataset = ChatbotDataset(train_data.values, tokenizer)

In [19]:
dataset[0]

(tensor([   1, 7335, 3372, 1211,    4,    2]),
 tensor([    1, 28336,  1240, 18159,    17,     2]))

In [20]:
# collate_fn 을 정의해줍니다.
# dataset으로부터 여러 item을 받아와 하나의 배치로 합칠 때, 어떻게 합칠지를 정의하는 부분입니다.

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
def collate_fn(batch):
    # batch: [(x1, y1), (x2, y2), ... (x4, y4)]
    # x는 x끼리, y는 y끼리
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(src_sample) # x는 x끼리
        tgt_batch.append(tgt_sample) # y는 y끼리
    
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX) # list -> torch tensor
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX) 
    return src_batch, tgt_batch # [sequence length, Batch size]

# dataloader 를 정의해줍니다.
batch_size = 32
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, 
                         num_workers=2, collate_fn=collate_fn,
                          pin_memory=True, drop_last=True)

In [21]:
x,y = next(iter(data_loader))

In [22]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer, loss_fn, train_dataloader):
    model.train()
    losses = 0

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        # (seq_length, batch_size)
        tgt_input = tgt[:-1,:] # 왜 target은 마지막 하나를 빼고 입력할까요?
        # <SOS> I love you

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()
        # tgt_out (seq_length, batch_size)
        tgt_out = tgt[1:, :] # 왜 첫번째 단어는 빼고 로스를 계산할까요?
        # I love you <EOS>

        # cross entropy: x (, class), y (*,)
        # logits (seq_length * batch_size, vocab_size)
        # tgt_out (seq_length * batch_size)
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), 
                       tgt_out.reshape(-1)) # softmax + negative loglikelihood loss
        loss.backward()
        optimizer.step()
        losses += loss.item()

    return losses / len(train_dataloader)

In [ ]:
EPOCHS = 20
for i in range(EPOCHS):
    epoch_loss = train_epoch(model, optimizer, loss_fn, data_loader)
    print('EPOCH {} LOSS {:.6f}'.format(i+1, epoch_loss))

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask) # model encoder 

    # ys: decoder input
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    # ys: [[1]] shape:(1,1)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1) 
        # [sequence length,  vocab size] 
        # ->  [batch size, sequence length, vocab size] 
        prob = model.generator(out[:, -1]) # generator: vocab사이즈로 만들어주는 선형변환
        _, next_word = torch.max(prob, dim=1) 
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        # shape: (6=sequence length, 1=batch size)
        if next_word == 2:
            break
    return ys

def predict(model, tokenizer, src_sentence):
    with torch.no_grad():
        model.eval()
        src = tokenizer.encode(src_sentence).ids
        src.insert(0,1)
        src.append(2)
        src = torch.tensor(src).long().unsqueeze(1)
        num_tokens = src.shape[0]
        src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
        tgt_tokens = greedy_decode(
            model,  src, src_mask, max_len=num_tokens + 5, start_symbol=1).flatten()
    return tokenizer.decode(tgt_tokens.cpu().tolist())

In [ ]:
predict(model,tokenizer,'잘 지내?')

In [ ]:
predict(model,tokenizer,'여행 가고 싶다.')

In [ ]:
while True:
  question = input('user1 : ')
  if question.lower == 'bye bye':
    break
  else:
    answer = predict(model,tokenizer, question)
    print('Bot' : {}'.format(answer))

# 과제 Quiz
오늘 과제는 아래 문제들을 푸는 것으로 대체하겠습니다.

1. nn.CrossEntropyLoss를 통해 loss function을 정의할 때, ignore_index 가 하는 역할은 무엇일까요?

2. Transformer 의 attention 구조가 기존 RNN 구조에 비해 갖는 장점은 무엇이 있을까요?

3. Transformer 디코더의 Masked multi-head attention에서 mask를 사용하는 이유가 무엇일까요?
